In [25]:
import numpy as np

# Outils sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Outils pytorch
import torch # package Racine
from torch import nn # Réseau de Neurones (Neural Network)
import torch.nn.functional as F # Couches, fonctions d'activation ...
import torch.autograd as autograd # Calcul dérivée (Gradient)
import torch.optim as optim # Optimiser pour la descente de Gradient

# Librairie graphique plotly

import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

In [26]:
class ClasseModele(nn.Module):

    def __init__(self, input_dim):
        super(ClasseModele, self).__init__()
        self.layer1 = nn.Linear(input_dim,50) # Première couche de 50 neurones
        self.layer2 = nn.Linear(50, 20)       # deuxième couche de 20 neurones
        self.layer3 = nn.Linear(20, 3)        # Couche de sortie de 3 neurones (3 classes)

    def forward(self, x):
        # https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x


In [27]:
class ClasseModele(nn.Module):

    def __init__(self, input_dim):
        super(ClasseModele, self).__init__()
        self.layer1 = nn.Linear(input_dim,50) # Première couche de 50 neurones
        self.layer2 = nn.Linear(50, 20)       # deuxième couche de 20 neurones
        self.layer3 = nn.Linear(20, 3)        # Couche de sortie de 3 neurones (3 classes)

    def forward(self, x):
        # https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

In [28]:
features, labels = load_iris(return_X_y=True)
print(features.shape,labels.shape)
iris = np.concatenate((features, labels.reshape([150,1])), axis=1)
iris[np.random.randint(len(iris), size=10)]

(150, 4) (150,)


array([[6.9, 3.1, 4.9, 1.5, 1. ],
       [6.8, 3.2, 5.9, 2.3, 2. ],
       [5.2, 3.4, 1.4, 0.2, 0. ],
       [6.1, 2.6, 5.6, 1.4, 2. ],
       [6.7, 3.3, 5.7, 2.5, 2. ],
       [6.4, 2.8, 5.6, 2.1, 2. ],
       [4.9, 3.6, 1.4, 0.1, 0. ],
       [6. , 2.9, 4.5, 1.5, 1. ],
       [6. , 2.2, 5. , 1.5, 2. ],
       [6. , 2.9, 4.5, 1.5, 1. ]])

In [29]:
features_train,features_dev, labels_train, labels_dev = train_test_split(features, labels, random_state=42)
print(features_train.shape,features_dev.shape, labels_train.shape, labels_dev.shape)
labels_dev[:10]

(112, 4) (38, 4) (112,) (38,)


array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1])

In [30]:
# Création d'un objet Modèle
model = ClasseModele(features_train.shape[1])

# choix de l'algorithme de Descente de Gradient et du learning Rate
# https://pytorch.org/docs/stable/optim.html#algorithms
optimizer = optim.Adam(model.parameters(), lr=0.01)

# choix de la fonction de coût
# https://pytorch.org/docs/stable/nn.html#loss-functions
loss_fn = nn.CrossEntropyLoss()

#Nombre d'itération sur les données
epochs = 100

print(model)

ClasseModele(
  (layer1): Linear(in_features=4, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=20, bias=True)
  (layer3): Linear(in_features=20, out_features=3, bias=True)
)


In [31]:
x_train, y_train = torch.from_numpy(features_train).float(), torch.from_numpy(labels_train).long()
for epoch in range(1, epochs+1):
    y_pred = model(x_train)
    loss = loss_fn(y_pred, y_train)
    if not epoch%10 :
        print('Epoch #%i Loss=%.4f'%(epoch,loss))

    optimizer.zero_grad() # Réinitialise le gradient
    loss.backward()       # Exécute la backpropagation
    optimizer.step()      # Met à jour les paramètres du réseau

Epoch #10 Loss=0.7902
Epoch #20 Loss=0.4522
Epoch #30 Loss=0.2739
Epoch #40 Loss=0.1334
Epoch #50 Loss=0.0858
Epoch #60 Loss=0.0725
Epoch #70 Loss=0.0676
Epoch #80 Loss=0.0653
Epoch #90 Loss=0.0638
Epoch #100 Loss=0.0627


In [32]:
#Création du Tensor x_dev, de type float
x_dev = torch.from_numpy(features_dev).float()
#Passage du modèle en mode info (important pour dropout !)
model.eval()
#Exécution du modèle
pred = model(x_dev)
#Sortie pred du computation graph, et conversion en tableau numpy
print(pred.grad_fn)
pred = pred.detach()
print(pred.grad_fn)
pred = pred.numpy()
pred

None


array([[ -7.3782697,   3.93165  ,  -1.398863 ],
       [ 13.558648 ,   5.6022086, -24.568176 ],
       [-23.389908 ,   1.141515 ,  16.812912 ],
       [ -7.927728 ,   3.6886394,  -0.5442437],
       [ -7.8797617,   4.2236967,  -1.5416119],
       [ 12.51717  ,   5.227301 , -22.752167 ],
       [ -2.588641 ,   4.0951357,  -6.0828485],
       [-13.693427 ,   2.858013 ,   5.610593 ],
       [-10.190494 ,   3.0888479,   2.5211372],
       [ -3.9768221,   4.063658 ,  -4.708779 ],
       [-12.209624 ,   3.0421357,   4.066594 ],
       [ 11.695863 ,   4.8120046, -21.069489 ],
       [ 13.351367 ,   5.4338307, -23.96554  ],
       [ 11.875048 ,   4.906565 , -21.444916 ],
       [ 12.958301 ,   5.285155 , -23.322258 ],
       [ -7.5839686,   4.068792 ,  -1.5944299],
       [-17.979183 ,   1.6631796,  11.346991 ],
       [ -4.46662  ,   3.8873441,  -3.8650136],
       [ -7.623141 ,   3.5835648,  -0.567894 ],
       [-17.823046 ,   1.5616091,  11.4357395],
       [ 11.494004 ,   4.79943  , -20.87

In [33]:
prevision_type_iris =  np.argmax(model(x_dev).detach().numpy(),axis=1)
prevision_type_iris

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0])

In [34]:
print ("Précision de la prévision = %.1f%% "%(accuracy_score(labels_dev, prevision_type_iris)*100))
print('%i prévision(s) correcte(s), %i erreur(s)'%(sum(prevision_type_iris==labels_dev),sum(prevision_type_iris!=labels_dev)))

Précision de la prévision = 100.0% 
38 prévision(s) correcte(s), 0 erreur(s)


In [35]:
acc_train=[]
acc_dev=[]
tab_epoch=[]
new_model = ClasseModele(features_train.shape[1])
new_optimizer = optim.Adam(new_model.parameters(), lr=0.01)
epochs = 100
for epoch in range(1, epochs+1):
    y_pred = new_model(x_train)
    loss = loss_fn(y_pred, y_train)

    if not epoch%10 :
        print('Epoch #%i Loss=%.2f'%(epoch,loss.item()))
        tab_epoch.append(epoch)
        acc_dev.append(
            accuracy_score(
                labels_dev,
                np.argmax(new_model(x_dev).detach().numpy(),axis=1)
            )*100
        )
        acc_train.append(
            accuracy_score(
                labels_train,
                np.argmax(new_model(x_train).detach().numpy(),axis=1)
            )*100
        )
    new_optimizer.zero_grad() # Réinitialise le gradient
    loss.backward()       # Exécute la backpropagation
    new_optimizer.step()      # Met à jours les paramètres du réseau

print(acc_train,acc_dev)


Epoch #10 Loss=0.66
Epoch #20 Loss=0.36
Epoch #30 Loss=0.17
Epoch #40 Loss=0.09
Epoch #50 Loss=0.07
Epoch #60 Loss=0.07
Epoch #70 Loss=0.06
Epoch #80 Loss=0.06
Epoch #90 Loss=0.06
Epoch #100 Loss=0.06
[65.17857142857143, 96.42857142857143, 97.32142857142857, 97.32142857142857, 98.21428571428571, 97.32142857142857, 97.32142857142857, 97.32142857142857, 97.32142857142857, 97.32142857142857] [71.05263157894737, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [36]:
train_line =go.Scatter(x=tab_epoch,y=acc_train,name='Train set')
dev_line =go.Scatter(x=tab_epoch,y=acc_dev,name='Dev set')

layout = go.Layout(title="Comparaison Précision Train/Test",titlefont=dict(size=40),autosize=False, width=1100,height=1100)

data=[train_line,dev_line]
fig = go.Figure(data=data, layout=layout)
fig.show()
